In [2]:
# Library imports and defining scaler object for normalisation

import os
import nibabel as nib
import numpy as np
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()



In [ ]:
path = "/content/drive/MyDrive/RAWDATA/MICCAI_BraTS2020_TrainingData" # Directory containing training data
total = (len(os.listdir(path)))
dir = os.listdir(path)
num = 0
for d in dir:
    num += 1
    curPath = os.path.join(path, d)
    # Flair
    flairPath = os.path.join(curPath, d+"_flair.nii")
    flairImage = nib.load(flairPath).get_fdata()
    scaledFlair = scaler.fit_transform(flairImage.reshape(-1, flairImage.shape[-1])).reshape(flairImage.shape)
    
    # T1CE
    t1cePath = os.path.join(curPath, d+"_t1ce.nii")
    t1ceImage = nib.load(t1cePath).get_fdata()
    scaledT1ce = scaler.fit_transform(t1ceImage.reshape(-1, t1ceImage.shape[-1])).reshape(t1ceImage.shape)

    #T1
    t1Path = os.path.join(curPath, d+"_t1.nii")
    t1Image = nib.load(t1Path).get_fdata()
    scaledT1 = scaler.fit_transform(t1Image.reshape(-1, t1Image.shape[-1])).reshape(t1Image.shape)

    # T2
    t2Path = os.path.join(curPath, d+"_t2.nii")
    t2Image = nib.load(t2Path).get_fdata()
    scaledT2 = scaler.fit_transform(t2Image.reshape(-1, t2Image.shape[-1])).reshape(t2Image.shape)

    
   

    # Mask
    maskPath = os.path.join(curPath, d + "_seg.nii.gz")
    maskImage = nib.load(maskPath).get_fdata()
    maskImage = maskImage.astype(np.uint8)
    maskImage[maskImage==4] = 3 # Reassign class label 4 to 3, as no image contains any voxels with class label 4

    # Combine Modalities

    combinedImage = np.stack([scaledFlair, scaledT1ce, scaledT1, scaledT2], axis=3)


    # Crop volumes to 128x128x128
    height = combinedImage.shape[0]
    hStart = int((height - 128) / 2)
    hEnd = int((height - ((height-128) / 2)))

    width = combinedImage.shape[1]
    wStart = int((width - 128) / 2)
    wEnd = int((width - ((width-128) / 2)))

    depth = combinedImage.shape[3]
    dStart = int((depth - 128) / 2)
    dEnd = int((depth - ((depth-128) / 2)))

    combinedImage = combinedImage[hStart:hEnd, wStart:wEnd, dStart:dEnd]
    maskImage = maskImage[hStart:hEnd, wStart:wEnd, dStart:dEnd]


    print(f"Saved: {num} out of {total}")

    # Save as numpy arrays
    imageSavePath = "/content/drive/MyDrive/VAL/image"
    imageSaveName = d + "_image"
    np.save(os.path.join(imageSavePath, imageSaveName), combinedImage)